In [3]:
import os
import sys

sys.path.append(os.path.abspath(".."))

In [8]:
import io
from typing import Dict
from uuid import uuid4

from smolagents import tools

import pandas as pd
import nc_py_api

from src.agent.adapters.tools.base import BaseTool

In [5]:
## monkey patching

tools.AUTHORIZED_TYPES = [
    "string",
    "boolean",
    "integer",
    "number",
    "image",
    "audio",
    "array",
    "object",
    "any",
    "null",
    "list",
    "dict",
    "dataframe",
]

In [24]:
class ExportData(BaseTool):
    name = "export_data"
    description = """Stores data in external cloud storage."""
    inputs = {
        "data": {"type": "dataframe", "description": "asset id data"},
    }
    outputs = {"url": {"type": "str", "description": "storage link"}}
    output_type = "dict"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(self, data: pd.DataFrame) -> Dict[str, str]:
        if data.empty:
            return {"url": None}

        nc = nc_py_api.Nextcloud(
            nextcloud_url=os.getenv("NX_URL"),
            nc_auth_user=os.getenv("NX_USER"),
            nc_auth_pass=os.getenv("NX_PASSWORD"),
        )

        file_name = f"agent/{str(uuid4())}.csv"

        buffer = io.BytesIO()

        # DataFrame als CSV in den BytesIO-Puffer schreiben
        data.to_csv(buffer, index=False)

        buffer.seek(0)
        nc.files.upload_stream(file_name, buffer)

        share = nc.files.sharing.create(
            path=file_name,  # path to the folder in your Nextcloud
            share_type=3,  # 3 = public link
        )
        return {"url": share.url}

In [25]:
kwargs = {"tools_api_base": "http://localhost:5050", "tools_api_limit": "100"}

In [26]:
tool = ExportData(**kwargs)

In [27]:
temp = pd.read_csv("/Users/steffen/dev/daily_data.csv")
temp.set_index("timestamp", inplace=True)

In [28]:
out = tool.forward(temp)

In [29]:
out

{'url': 'https://nx74354.your-storageshare.de/s/gd2EwSpESJNK4jf'}